***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-17c1  GCE       2                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle 
import pandas as pd
import numpy as np
import google.cloud
from google.cloud import storage
import hashlib
from typing import Dict,List,Tuple,FrozenSet


In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 14 17:19 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
sc.addFile("/home/dataproc/inverted_index_proj.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '209383306irass3' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith('multi'):
        paths.append(full_path+b.name)

#TOKENIZER

In [9]:
nltk.download('stopwords')
nltk_stopwords = frozenset(stopwords.words('english'))
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){,24}""", re.UNICODE)
NUM_BUCKETS = 124

def get_stopwords() -> FrozenSet[str]:
    corpus_stopwords = ["category", "references", "also", "external", "links",
                        "may", "first", "see", "history", "people", "one", "two",
                        "part", "thumb", "including", "second", "following",
                        "many", "however", "would", "became"]  # TODO
    return nltk_stopwords.union(corpus_stopwords)

def tokenize(text: str) -> List[str]:
    stopwords = get_stopwords()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    return [token for token in tokens if token not in stopwords]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#PREPROCESS DATA

In [10]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()


def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS


def read_gcp_files(paths: List[str], index_type: str):
    parquet_file = spark.read.parquet(*paths)
    num_of_docs = parquet_file.count()
    return parquet_file.select(index_type, "id").rdd, num_of_docs

def reduce_word_counts(word_counts: List[Tuple]) -> List[Tuple]:
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    return sorted(word_counts)

def calculate_df(postings, as_dict: bool = True):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    w2df = postings.mapValues(len)
    if as_dict:
        return w2df.collectAsMap()
    return w2df

def calculate_idf(postings, num_of_docs: int, as_dict: bool = True):
    w2idf = postings.mapValues(lambda term: np.log10(num_of_docs / len(term)))
    if as_dict:
        return w2idf.collectAsMap()
    return w2idf

def calculate_tf_idf(postings, idf, as_dict: bool = True):
    tf_idf = postings.map(lambda token: (token[0], [(doc[0], doc[1] * idf[token[0]]) for doc in token[1]]))
    if as_dict is True:
        return tf_idf.collectAsMap()
    return tf_idf

def calculate_dl(doc_text_pairs, index_type: str, as_dict: bool = True):
    dl = doc_text_pairs.map(lambda x: (x['id'], len(tokenize(x[index_type]))))
    if as_dict is True:
        return dl.collectAsMap()
    return dl
    
def calculate_term_total(postings, as_dict: bool = True):
    term_total = postings.mapValues(lambda term: np.sum([x[1] for x in term]))
    if as_dict:
        return term_total.collectAsMap()
    return term_total

def partition_postings_and_write(postings, bucket_name: str, index_name: str):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''
    rdd = postings.map(lambda x: (token2bucket_id(x[0]), [x])).reduceByKey(lambda x, y: x + y)
    posting_locs_rdd = rdd.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name, index_name))
    return posting_locs_rdd

def word_count(text: str, doc_id: int, stemming: bool) -> List[Tuple]:
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = tokenize(text)
    if stemming:
        counter = Counter([stem(token) for token in tokens])
    else:
        counter = Counter(tokens)
    return [(token, (doc_id, count)) for (token, count) in counter.items()]

def get_super_posting_locs(bucket_name) -> defaultdict:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
    return super_posting_locs

def process(doc_text_pairs, index_type: str, stemming: bool = False, anchor: bool = False):
    if anchor:
        doc_text_pairs = doc_text_pairs.map(lambda x: x[0]).flatMap(lambda x: x)
        index_type = 'text'
    word_counts = doc_text_pairs.flatMap(lambda x: word_count(text=x[index_type], doc_id=x['id'], stemming=stemming))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    if anchor:
        postings = postings.mapValues(lambda x: list(set(x)))
    return postings

#PICKLE

In [11]:

bucket = client.bucket(bucket_name)

def download_from_gcp(source_path: str, destination_path: str) -> None:
    blob = bucket.get_blob(source_path)
    blob.download_to_filename(destination_path)

def get_index(index_name: str):
    source_path = f'postings_{index_name}/index.pkl'
    destination_path = f'{index_name}.pkl'
    download_from_gcp(source_path=source_path, destination_path=destination_path)
    return InvertedIndex().read_index(base_dir='.', name=destination_path)

def write_pickle_file(path: str, file) -> None:
    with open(path, 'wb') as f:
        pkl.dump(file, f)

def read_pickle_file(path: str):
    with open(path, 'rb') as f:
        file = pkl.loads(f.read())
    return file

def move_to_gcp(src, dest):
    !gsutil cp $src $dest

In [12]:
from inverted_index_proj import InvertedIndex

In [13]:

def indexsize_file(pand, invertind, num_of_docs):
    start = 0
    jump = 5000
    end = jump
    leng = num_of_docs
    perish_flag = False
    while True:
        for row in pand.iloc[start:end].iterrows():
            tokens = tokenize(row[1]['text'])
            invertind.add_doc(row[1]['id'], tokens)
        if perish_flag:
            break
        print(end)
        start += jump
        end += jump
        if end> leng:
            end = leng
            perish_flag = True

            
            
def index_all(paths, invertind):
    for path in paths:
        rdd, num_of_docs  = read_gcp_files([path], 'text')
        pand = rdd.toDF().toPandas()
        if num_of_docs > 100000:
            indexsize_file(pand.iloc[0:100000], invertind, 100000)
            indexsize_file(pand.iloc[100000:], invertind, (num_of_docs-100000))
        else:
            indexsize_file(pand, invertind, num_of_docs)
        del pand
        print("finished a file")

In [ ]:
invertind = InvertedIndex.read_index("/home/dataproc", "body_index")

In [ ]:
def rare_words_finder(index : InvertedIndex, treshhold : int = 50 ) -> List[str]:
    

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
finished a file


5000
10000
15000
finished a file


In [56]:
invertind.write_index("/home/dataproc", "indexsave")

In [ ]:
index_types = [('anchor_text',False)]
base_path = f'gs://{bucket_name}/'
for index in index_types:
      index_type,stemming = index  
      index_name = index_type
      anchor = index_type == 'anchor_text'
      doc_text_pairs, num_of_docs = read_gcp_files(paths, index_type)
      if index_type == 'title':
            title_id_dict = doc_text_pairs.map(lambda x: {x['id']: x[index_type]}).collect()
            write_pickle_file('title.pkl', title_id_dict)
      if anchor:
            doc_text_pairs = doc_text_pairs.map(lambda x: x[0]).flatMap(lambda x: x)
            index_type = 'text'
      word_counts = doc_text_pairs.flatMap(lambda x: word_count(text=x[index_type], doc_id=x['id'], stemming=False))
      postings = word_counts.groupByKey().mapValues(reduce_word_counts)
        
      if anchor:
            postings = postings.mapValues(lambda x: list(set(x)))
      postings = postings.filter(lambda x: len(x[1]) >= 50).collect()
      posting_locs_list = partition_postings_and_write(postings, bucket_name, index_name).collect()
      super_posting_locs = get_super_posting_locs(bucket_name)
      inverted_index = InvertedIndex()
      inverted_index.posting_locs = super_posting_locs
      inverted_index.df = calculate_df(postings)
      if index_type == 'title' or index_name == 'text':
        if index_type == 'text':
            idf_dict = calculate_idf(postings, num_of_docs)
            idf_path = f'tf_idf_{index_name}.pkl'
            write_pickle_file(idf_path, idf_dict)
            move_to_gcp(idf_path, base_path + idf_path)
            inverted_index.term_total = calculate_term_total(postings)
            inverted_index.num_of_docs = num_of_docs
        dl_dict = calculate_dl(doc_text_pairs, index_type)
        dl_path = f'dl_{index_name}.pkl'
        write_pickle_file(dl_path, dl_dict)
        move_to_gcp(dl_path, base_path + dl_path)
      inverted_index.write_index('.', 'index')
      index_src = "index.pkl"
      index_dst = base_path + f'postings_{index_name}/{index_src}'
      move_to_gcp(index_src, index_dst)
  


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag